# TP noté de Machine Learning, durée 2h
## Le notebook est à envoyer par mail à l'adresse suivante :
### abdallah.elhidali@gmail.com   ---> mettez votre nom-prénom dans l'objet du mail
## ❗️ **_Il est strictement interdit d'utiliser des outils d'IA générative comme ChatGPT ou autres (tout étudiant ayant utilisé ces outils sera éliminé de ce TP noté)._** ❗️
----
## Apprentissage Supervisé
## Projet : Recherche de Donateurs pour *CharityOrg*

Dans ce notebook, votre tâche sera d'implémenter les fonctionnalités supplémentaires nécessaires pour compléter ce projet avec succès. Les sections qui commencent par **'Implémentation'** dans l'en-tête indiquent que le bloc de code suivant nécessitera des fonctionnalités supplémentaires que vous devez fournir. Des instructions seront fournies pour chaque section et les détails de l'implémentation sont marqués dans le bloc de code avec une mention `'TODO'`. Veuillez lire attentivement les instructions !

En plus d'implémenter du code, il y aura des questions auxquelles vous devrez répondre concernant le projet et votre implémentation. Chaque section où vous devrez répondre à une question est précédée d'un en-tête **'Question X'**. Lisez attentivement chaque question et fournissez des réponses complètes dans les zones de texte qui commencent par **'Réponse :'**. Votre projet sera évalué en fonction de vos réponses à chaque question et de l'implémentation que vous fournirez. **Vos réponses doivent se baser sur ce qu'on a vu pendant les cours précédents.**

>**Note :** Les cellules de code et Markdown peuvent être exécutées en utilisant le raccourci clavier **Shift + Enter**. De plus, les cellules Markdown peuvent être éditées en double-cliquant généralement sur la cellule pour entrer en mode édition.

## Pour Commencer

Dans ce projet, vous utiliserez plusieurs algorithmes d'apprentissage supervisé de votre choix pour modéliser avec précision les revenus des individus en utilisant les données collectées lors du recensement américain de 1994. Vous choisirez ensuite le meilleur algorithme candidat à partir des résultats préliminaires et optimiserez davantage cet algorithme pour mieux modéliser les données. Votre objectif avec cette implémentation est de construire un modèle qui prédit avec précision si un individu gagne plus de 50 000 $. Ce type de tâche peut survenir dans un contexte associatif, où les organisations survivent grâce aux dons. Comprendre le revenu d'un individu peut aider une association à mieux comprendre quelle importance de don demander, ou s'il faut même prendre contact pour commencer. Bien qu'il puisse être difficile de déterminer directement la tranche de revenus générale d'un individu à partir de sources publiques, nous pouvons (comme nous le verrons) déduire cette valeur à partir d'autres caractéristiques publiquement disponibles.


----
## Explorer les Données
Exécutez la cellule de code ci-dessous pour charger les bibliothèques Python nécessaires et charger les données du recensement. Notez que la dernière colonne de cet ensemble de données, `'income'`, sera notre étiquette cible (si un individu gagne plus de 50 000 $ annuellement ou non). Toutes les autres colonnes sont des caractéristiques sur chaque individu dans la base de données du recensement.

In [ ]:
# Importer les bibliothèques nécessaires pour ce projet
import numpy as np
import pandas as pd
from time import time
from IPython.display import display # Permet l'utilisation de display() pour les DataFrames

# Importer le code de visualisation supplémentaire visuals.py
import visuals as vs

# Affichage amélioré pour les notebooks
%matplotlib inline

# Charger l'ensemble de données du recensement
data = pd.read_csv("census.csv")

# Succès - Afficher le premier enregistrement
display(data.head(n=1))

### Implémentation : Exploration des Données
Une investigation rapide de l'ensemble de données déterminera combien d'individus appartiennent à chaque groupe et nous renseignera sur le pourcentage de ces individus gagnant plus de 50 000 $. Dans la cellule de code ci-dessous, vous devrez calculer ce qui suit :
- Le nombre total d'enregistrements, `'n_records'`
- Le nombre d'individus gagnant plus de 50 000 $ par an, `'n_greater_50k'`.
- Le nombre d'individus gagnant au plus 50 000 $ par an, `'n_at_most_50k'`.
- Le pourcentage d'individus gagnant plus de 50 000 $ par an, `'greater_percent'`.

** ASTUCE : ** Vous devrez peut-être regarder le tableau ci-dessus pour comprendre comment les entrées `'income'` sont formatées. 

In [ ]:
# TODO : Nombre total d'enregistrements
n_records = None

# TODO : Nombre d'enregistrements où le revenu de l'individu est supérieur à 50 000 $
n_greater_50k = None

# TODO : Nombre d'enregistrements où le revenu de l'individu est au plus 50 000 $
n_at_most_50k = None

# TODO : Pourcentage d'individus dont le revenu est supérieur à 50 000 $
greater_percent = None

# Afficher les résultats
print("Nombre total d'enregistrements : {}".format(n_records))
print("Individus gagnant plus de 50 000 $ : {}".format(n_greater_50k))
print("Individus gagnant au plus 50 000 $ : {}".format(n_at_most_50k))
print("Pourcentage d'individus gagnant plus de 50 000 $ : {}%".format(greater_percent))

## Préparation des Données
Avant que les données puissent être utilisées comme entrée pour les algorithmes d'apprentissage automatique, elles doivent souvent être nettoyées, formatées et restructurées — c'est ce qu'on appelle généralement le **prétraitement**. Heureusement, pour cet ensemble de données, il n'y a pas d'entrées invalides ou manquantes à traiter, cependant, il y a certaines qualités concernant certaines caractéristiques qui doivent être ajustées. Ce prétraitement peut grandement aider avec le résultat et la puissance prédictive de presque tous les algorithmes d'apprentissage.

### Transformation des Caractéristiques Continues Asymétriques
Un ensemble de données peut parfois contenir au moins une caractéristique dont les valeurs ont tendance à se situer près d'un seul nombre, mais qui aura aussi un nombre non négligeable de valeurs beaucoup plus grandes ou plus petites que ce nombre unique. Les algorithmes peuvent être sensibles à de telles distributions de valeurs et peuvent sous-performer si la plage n'est pas correctement normalisée. Dans l'ensemble de données du recensement, deux caractéristiques correspondent à cette description : `capital-gain` et `capital-loss`.

Exécutez la cellule de code ci-dessous pour tracer un histogramme de ces deux caractéristiques. Notez la plage des valeurs présentes et comment elles sont distribuées.

In [ ]:
# Diviser les données en caractéristiques et étiquette cible
income_raw = data['income']
features_raw = data.drop('income', axis = 1)

# Visualiser les caractéristiques continues asymétriques des données originales
vs.distribution(data)

Pour les distributions de caractéristiques fortement asymétriques comme `'capital-gain'` et `'capital-loss'`, il est courant d'appliquer une <a href="https://en.wikipedia.org/wiki/Data_transformation_(statistics)">transformation logarithmique</a> sur les données afin que les valeurs très grandes et très petites n'affectent pas négativement la performance d'un algorithme d'apprentissage. L'utilisation d'une transformation logarithmique réduit significativement la plage des valeurs causée par les valeurs aberrantes. Il faut cependant faire attention lors de l'application de cette transformation : le logarithme de `0` n'est pas défini, nous devons donc traduire les valeurs par une petite quantité au-dessus de `0` pour appliquer le logarithme avec succès.

Exécutez la cellule de code ci-dessous pour effectuer une transformation sur les données et visualiser les résultats. Encore une fois, notez la plage des valeurs et comment elles sont distribuées.

In [ ]:
# Log-transform the skewed features
skewed = ['capital-gain', 'capital-loss']
features_log_transformed = pd.DataFrame(data = features_raw)
features_log_transformed[skewed] = features_raw[skewed].apply(lambda x: np.log(x + 1))

# Visualize the new log distributions
vs.distribution(features_log_transformed, transformed = True)

### Normalisation des Caractéristiques Numériques
En plus d'effectuer des transformations sur les caractéristiques fortement asymétriques, il est souvent recommandé d'effectuer un certain type de mise à l'échelle sur les caractéristiques numériques. L'application d'une mise à l'échelle aux données ne modifie pas la forme de la distribution de chaque caractéristique (comme `'capital-gain'` ou `'capital-loss'` ci-dessus) ; cependant, la normalisation garantit que chaque caractéristique est traitée de manière égale lors de l'application des algorithmes d'apprentissage supervisé. Notez qu'une fois la mise à l'échelle appliquée, l'observation des données sous leur forme brute n'aura plus la même signification originale, comme illustré ci-dessous.

Exécutez la cellule de code ci-dessous pour normaliser chaque caractéristique numérique. Nous utiliserons [`sklearn.preprocessing.MinMaxScaler`](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html) pour cela.

In [ ]:
# Import sklearn.preprocessing.StandardScaler
from sklearn.preprocessing import MinMaxScaler

# Initialize a scaler, then apply it to the features
scaler = MinMaxScaler() # default=(0, 1)
numerical = ['age', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']

features_log_minmax_transform = pd.DataFrame(data = features_log_transformed)
features_log_minmax_transform[numerical] = scaler.fit_transform(features_log_transformed[numerical])

# Show an example of a record with scaling applied
display(features_log_minmax_transform.head(n = 5))

### Implémentation : Prétraitement des Données

D'après le tableau dans **Explorer les Données** ci-dessus, nous pouvons voir qu'il y a plusieurs caractéristiques pour chaque enregistrement qui sont non numériques. Généralement, les algorithmes d'apprentissage s'attendent à ce que les entrées soient numériques, ce qui nécessite que les caractéristiques non numériques (appelées *variables catégorielles*) soient converties. Une méthode populaire pour convertir les variables catégorielles est d'utiliser le codage **one-hot encoding**. Le one-hot encoding crée une variable _"fictive"_ pour chaque catégorie possible de chaque caractéristique non numérique. Par exemple, supposons que `someFeature` a trois entrées possibles : `A`, `B`, ou `C`. Nous encodons alors cette caractéristique en `someFeature_A`, `someFeature_B` et `someFeature_C`.

|   | someFeature |                    | someFeature_A | someFeature_B | someFeature_C |
| :-: | :-: |                            | :-: | :-: | :-: |
| 0 |  B  |  | 0 | 1 | 0 |
| 1 |  C  | ----> encodage one-hot ----> | 0 | 0 | 1 |
| 2 |  A  |  | 1 | 0 | 0 |

De plus, comme pour les caractéristiques non numériques, nous devons convertir l'étiquette cible non numérique `'income'` en valeurs numériques pour que l'algorithme d'apprentissage fonctionne. Puisqu'il n'y a que deux catégories possibles pour cette étiquette ("<=50K" et ">50K"), nous pouvons éviter d'utiliser le one-hot encoding et simplement encoder ces deux catégories comme `0` et `1`, respectivement. Dans la cellule de code ci-dessous, vous devrez implémenter ce qui suit :
 - Utiliser [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) pour effectuer le one-hot encoding sur les données `'features_log_minmax_transform'`.
 - Convertir l'étiquette cible `'income_raw'` en entrées numériques.
   - Définir les enregistrements avec "<=50K" à `0` et les enregistrements avec ">50K" à `1`.

In [ ]:
# TODO: One-hot encode the 'features_log_minmax_transform' data using pandas.get_dummies()
features_final = None

# TODO: Encode the 'income_raw' data to numerical values
income = None

# Print the number of features after one-hot encoding
encoded = list(features_final.columns)
print("{} total features after one-hot encoding.".format(len(encoded)))

# Uncomment the following line to see the encoded feature names
# print(encoded)

### Mélange et Division des Données
Maintenant que toutes les _variables catégorielles_ ont été converties en caractéristiques numériques, et que toutes les caractéristiques numériques ont été normalisées. Comme toujours, nous allons maintenant diviser les données (à la fois les caractéristiques et leurs étiquettes) en ensembles d'entraînement et de test. 80% des données seront utilisées pour l'entraînement et 20% pour les tests.

Exécutez la cellule de code ci-dessous pour effectuer cette division.

In [ ]:
# Import train_test_split
from sklearn.model_selection import train_test_split

# Split the 'features' and 'income' data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_final, 
                                                    income, 
                                                    test_size = 0.2, 
                                                    random_state = 0)

# Show the results of the split
print("Training set has {} samples.".format(X_train.shape[0]))
print("Testing set has {} samples.".format(X_test.shape[0]))

----
## Évaluation de la Performance du Modèle
Dans cette section, nous allons étudier quatre algorithmes différents et déterminer lequel est le plus performant pour modéliser les données. Trois de ces algorithmes seront des algorithmes d'apprentissage supervisé de votre choix, et le quatrième algorithme est connu comme un *prédicteur naïf*.

### Métriques et le Prédicteur Naïf
`CharityOrg`, grâce à ses recherches, sait que les individus qui gagnent plus de 50 000$ sont les plus susceptibles de faire un don à leur association. Pour cette raison, `CharityOrg` est particulièrement intéressé à prédire avec précision qui gagne plus de 50 000$. Il semblerait que l'utilisation de l'**exactitude** comme métrique pour évaluer la performance d'un modèle particulier serait appropriée. De plus, identifier quelqu'un qui ne gagne `pas` plus de 50 000$ comme quelqu'un qui le fait serait préjudiciable pour `CharityOrg`, puisqu'ils cherchent à trouver des individus disposés à faire des dons. Par conséquent, la capacité d'un modèle à prédire précisément ceux qui gagnent plus de 50 000$ est `plus importante` que la capacité du modèle à **rappeler** ces individus. Nous pouvons utiliser le **score F-bêta** comme métrique qui prend en compte à la fois la précision et le rappel :

$$ F_{\beta} = (1 + \beta^2) \cdot \frac{precision \cdot recall}{\left( \beta^2 \cdot precision \right) + recall} $$

En particulier, lorsque $\beta = 0.5$, plus d'importance est accordée à la précision. C'est ce qu'on appelle le **score $F_{0.5}$** (ou score F par simplicité).

En regardant la distribution des classes `"ceux qui gagnent au plus 50 000$, et ceux qui gagnent plus`, il est clair que la plupart des individus ne gagnent pas plus de 50 000$. 
Cela peut grandement affecter l'**exactitude**, puisque nous pourrions simplement dire `"cette personne ne gagne pas plus de 50 000$"` et généralement avoir raison, sans même regarder les données ! Faire une telle déclaration serait qualifié de **naïf**, puisque nous n'avons considéré aucune information pour étayer cette affirmation. Il est toujours important de considérer la `prédiction naïve` pour vos données, pour aider à établir une référence permettant de déterminer si un modèle performe bien. Cela étant dit, utiliser cette prédiction serait inutile : Si nous prédisions que toutes les personnes gagnaient moins de 50 000$, `CharityOrg` n'identifierait personne comme donateur.

#### Note : Récapitulatif de l'exactitude, la précision, le rappel

L'**Exactitude** mesure la fréquence à laquelle le classificateur fait la bonne prédiction. C'est le rapport entre le nombre de prédictions correctes et le nombre total de prédictions (le nombre de points de données de test).

La **Précision** nous indique quelle proportion des messages que nous avons classés comme spam étaient effectivement du spam.
C'est un ratio des vrais positifs (mots classés comme spam et qui sont effectivement du spam) sur tous les positifs (tous les mots classés comme spam, indépendamment de si c'était la bonne classification), en d'autres termes, c'est le ratio :

`[Vrais Positifs/(Vrais Positifs + Faux Positifs)]`

Le **Rappel (sensibilité)** nous indique quelle proportion des messages qui étaient effectivement du spam ont été classés par nous comme spam.
C'est un ratio des vrais positifs (mots classés comme spam et qui sont effectivement du spam) sur tous les mots qui étaient effectivement du spam, en d'autres termes, c'est le ratio :

`[Vrais Positifs/(Vrais Positifs + Faux Négatifs)]`

Pour les problèmes de classification qui sont déséquilibrés dans leurs distributions de classification comme dans notre cas, par exemple si nous avions 100 messages texte et que seulement 2 étaient du spam et les 98 autres non, l'exactitude seule n'est pas une très bonne métrique. Nous pourrions classer 90 messages comme non spam (y compris les 2 qui étaient du spam mais que nous classons comme non spam, donc ce seraient des faux négatifs) et 10 comme spam (tous les 10 faux positifs) et obtenir quand même un score d'exactitude raisonnablement bon. Pour de tels cas, la précision et le rappel sont très utiles. Ces deux métriques peuvent être combinées pour obtenir le score F1, qui est la moyenne pondérée (moyenne harmonique) des scores de précision et de rappel. Ce score peut varier de 0 à 1, 1 étant le meilleur score F1 possible (nous prenons la moyenne harmonique car nous traitons des ratios).

### Question 1 - Performance du Prédicteur Naïf
* Si nous choisissions un modèle qui prédisait toujours qu'un individu gagnait plus de 50 000$, quelle serait l'exactitude et le score F de ce modèle sur cet ensemble de données ? Vous devez utiliser la cellule de code ci-dessous et attribuer vos résultats à `'accuracy'` et `'fscore'` pour une utilisation ultérieure.

** Veuillez noter ** que l'objectif de générer un prédicteur naïf est simplement de montrer à quoi ressemblerait un modèle de base sans aucune intelligence. Dans le monde réel, idéalement, votre modèle de base serait soit les résultats d'un modèle précédent, soit basé sur un article de recherche que vous cherchez à améliorer. Lorsqu'il n'y a pas de modèle de référence établi, obtenir un résultat meilleur qu'un choix aléatoire est un point de départ possible.

** ASTUCE : ** 

* Lorsque nous avons un modèle qui prédit toujours '1' (c'est-à-dire que l'individu gagne plus de 50k), notre modèle n'aura pas de Vrais Négatifs (TN) ou de Faux Négatifs (FN) car nous ne faisons aucune prédiction négative (valeur '0'). Par conséquent, notre Exactitude dans ce cas devient la même que notre Précision (Vrais Positifs/(Vrais Positifs + Faux Positifs)) car chaque prédiction que nous avons faite avec la valeur '1' qui aurait dû être '0' devient un Faux Positif ; donc notre dénominateur dans ce cas est le nombre total d'enregistrements que nous avons au total.
* Notre score de Rappel (Vrais Positifs/(Vrais Positifs + Faux Négatifs)) dans ce contexte devient 1 car nous n'avons pas de Faux Négatifs.

In [ ]:
# TODO: Calculate accuracy, precision and recall
accuracy = None
recall = None
precision = None

# TODO: Calculate F-score using the formula above for beta = 0.5 and correct values for precision and recall.
fscore = None

# Print the results 
print("Naive Predictor: [Accuracy score: {:.4f}, F-score: {:.4f}]".format(accuracy, fscore))

### Modèles d'Apprentissage Supervisé
**Voici quelques-uns des modèles d'apprentissage supervisé actuellement disponibles dans** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **parmi lesquels vous pouvez choisir :**
- Naive Bayes Gaussien (GaussianNB)
- Arbres de Décision
- Méthodes d'Ensemble (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K Plus Proches Voisins (KNeighbors)
- Classificateur par Descente de Gradient Stochastique (SGDC)
- Machines à Vecteurs de Support (SVM)
- Régression Logistique

### Question 2 - Application du Modèle
Listez trois des modèles d'apprentissage supervisé ci-dessus qui sont appropriés pour ce problème et que vous testerez sur les données du recensement. Pour chaque modèle choisi

- Décrivez une application réelle dans l'industrie où le modèle peut être appliqué.
- Quelles sont les forces du modèle ; quand performe-t-il bien ?
- Quelles sont les faiblesses du modèle ; quand performe-t-il mal ?
- Qu'est-ce qui fait de ce modèle un bon candidat pour le problème, étant donné ce que vous savez sur les données ?

**ASTUCE :**

Structurez votre réponse dans le même format que ci-dessus, avec 4 parties pour chacun des trois modèles que vous choisissez.

**Réponse :**

### Implémentation - Création d'un Pipeline d'Entraînement et de Prédiction
Pour évaluer correctement la performance de chaque modèle que vous avez choisi, il est important de créer un pipeline d'entraînement et de prédiction qui vous permet d'entraîner rapidement et efficacement des modèles en utilisant différentes tailles de données d'entraînement et d'effectuer des prédictions sur les données de test. Votre implémentation ici sera utilisée dans la section suivante.
Dans le bloc de code ci-dessous, vous devrez implémenter ce qui suit :
 - Importer `fbeta_score` et `accuracy_score` de [`sklearn.metrics`](http://scikit-learn.org/stable/modules/classes.html#sklearn-metrics-metrics).
 - Entraîner l'apprenant sur les données d'entraînement échantillonnées et enregistrer le temps d'entraînement.
 - Effectuer des prédictions sur les données de test `X_test`, et aussi sur les 300 premiers points d'entraînement `X_train[:300]`.
   - Enregistrer le temps total de prédiction.
 - Calculer le score d'exactitude pour le sous-ensemble d'entraînement et l'ensemble de test.
 - Calculer le score F pour le sous-ensemble d'entraînement et l'ensemble de test.
   - Assurez-vous de définir le paramètre `beta` !

In [ ]:
# TODO: Import two metrics from sklearn - fbeta_score and accuracy_score

def train_predict(learner, sample_size, X_train, y_train, X_test, y_test): 
    '''
    inputs:
       - learner: the learning algorithm to be trained and predicted on
       - sample_size: the size of samples (number) to be drawn from training set
       - X_train: features training set
       - y_train: income training set
       - X_test: features testing set
       - y_test: income testing set
    '''
    
    results = {}
    
    # TODO: Fit the learner to the training data using slicing with 'sample_size' using .fit(training_features[:], training_labels[:])
    start = time() # Get start time
    learner = None
    end = time() # Get end time
    
    # TODO: Calculate the training time
    results['train_time'] = None
        
    # TODO: Get the predictions on the test set(X_test),
    #       then get predictions on the first 300 training samples(X_train) using .predict()
    start = time() # Get start time
    predictions_test = None
    predictions_train = None
    end = time() # Get end time
    
    # TODO: Calculate the total prediction time
    results['pred_time'] = None
            
    # TODO: Compute accuracy on the first 300 training samples which is y_train[:300]
    results['acc_train'] = None
        
    # TODO: Compute accuracy on test set using accuracy_score()
    results['acc_test'] = None
    
    # TODO: Compute F-score on the the first 300 training samples using fbeta_score()
    results['f_train'] = None
        
    # TODO: Compute F-score on the test set which is y_test
    results['f_test'] = None
       
    # Success
    print("{} trained on {} samples.".format(learner.__class__.__name__, sample_size))
        
    # Return the results
    return results

### Implémentation : Évaluation Initiale du Modèle
Dans la cellule de code, vous devrez implémenter ce qui suit :
- Importer les trois modèles d'apprentissage supervisé dont vous avez discuté dans la section précédente.
- Initialiser les trois modèles et les stocker dans `'clf_A'`, `'clf_B'`, et `'clf_C'`.
  - Utiliser un `'random_state'` pour chaque modèle que vous utilisez, si fourni.
  - **Note :** Utiliser les paramètres par défaut pour chaque modèle — vous ajusterez un modèle spécifique dans une section ultérieure.
- Calculer le nombre d'enregistrements égal à 1%, 10% et 100% des données d'entraînement.
  - Stocker ces valeurs respectivement dans `'samples_1'`, `'samples_10'`, et `'samples_100'`.

**Note :** Selon les algorithmes que vous avez choisis, l'implémentation suivante peut prendre un certain temps à s'exécuter !

In [ ]:
# TODO: Import the three supervised learning models from sklearn

# TODO: Initialize the three models
clf_A = None
clf_B = None
clf_C = None

# TODO: Calculate the number of samples for 1%, 10%, and 100% of the training data
samples_100 = None
samples_10 = None
samples_1 = None

# Collect results on the learners
results = {}
for clf in [clf_A, clf_B, clf_C]:
    clf_name = clf.__class__.__name__
    results[clf_name] = {}
    for i, samples in enumerate([samples_1, samples_10, samples_100]):
        results[clf_name][i] = \
        train_predict(clf, samples, X_train, y_train, X_test, y_test)

# Run metrics visualization for the three supervised learning models chosen
vs.evaluate(results, accuracy, fscore)

----
## Amélioration des Résultats
Dans cette dernière section, vous choisirez parmi les trois modèles d'apprentissage supervisé le *meilleur* modèle à utiliser sur les données étudiées. Vous effectuerez ensuite une optimisation par recherche sur grille pour le modèle sur l'ensemble complet d'entraînement (`X_train` et `y_train`) en ajustant au moins un paramètre pour améliorer le F-score du modèle non ajusté.

### Question 3 - Choisir le Meilleur Modèle

* Sur la base de l'évaluation que vous avez effectuée précédemment, expliquez en un ou deux paragraphes à *CharityOrg* lequel des trois modèles vous pensez être le plus approprié pour la tâche d'identification des individus qui gagnent plus de 50 000$.

**ASTUCE :**
Regardez le graphique en bas à gauche de la cellule ci-dessus (la visualisation créée par `vs.evaluate(results, accuracy, fscore)`) et vérifiez le F-score pour l'ensemble de test lorsque 100% de l'ensemble d'entraînement est utilisé. Quel modèle a le score le plus élevé ? Votre réponse devrait inclure une discussion sur :
* les métriques - F-score sur le test lorsque 100% des données d'entraînement sont utilisées.
* le temps de prédiction/entraînement.
* l'adéquation de l'algorithme aux données.

**Réponse :**


### Question 4 - Description du Modèle en Termes Simples

* En un ou deux paragraphes, expliquez à *CharityOrg*, en termes simples, comment le modèle final choisi est censé fonctionner. Assurez-vous de décrire les principales qualités du modèle, comme la façon dont le modèle est entraîné et comment il fait une prédiction. Évitez d'utiliser du jargon mathématique avancé, comme la description d'équations.



**Réponse :**


### Implémentation : Ajustement du Modèle
Affinez le modèle choisi. Utilisez la recherche sur grille (`GridSearchCV`) avec au moins un paramètre important ajusté avec au moins 3 valeurs différentes. Vous devrez utiliser l'ensemble complet d'entraînement pour cela. Dans la cellule de code ci-dessous, vous devrez implémenter ce qui suit :
- Importer [`sklearn.grid_search.GridSearchCV`](http://scikit-learn.org/0.17/modules/generated/sklearn.grid_search.GridSearchCV.html) et [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Initialiser le classificateur que vous avez choisi et le stocker dans `clf`.
 - Définir un `random_state` si disponible au même état que vous avez défini précédemment.
- Créer un dictionnaire des paramètres que vous souhaitez ajuster pour le modèle choisi.
 - Exemple : `parameters = {'parameter' : [liste de valeurs]}`.
 - **Note :** Évitez d'ajuster le paramètre `max_features` de votre apprenant si ce paramètre est disponible !
- Utiliser `make_scorer` pour créer un objet de score `fbeta_score` (avec $\beta = 0.5$).
- Effectuer une recherche sur grille sur le classificateur `clf` en utilisant le `'scorer'`, et le stocker dans `grid_obj`.
- Ajuster l'objet de recherche sur grille aux données d'entraînement (`X_train`, `y_train`), et le stocker dans `grid_fit`.

**Note :** Selon l'algorithme choisi et la liste des paramètres, l'implémentation suivante peut prendre un certain temps à s'exécuter !

In [ ]:
# TODO: Import 'GridSearchCV', 'make_scorer', and any other necessary libraries

# TODO: Initialize the classifier
clf = None

# TODO: Create the parameters list you wish to tune, using a dictionary if needed.
# HINT: parameters = {'parameter_1': [value1, value2], 'parameter_2': [value1, value2]}
parameters = None

# TODO: Make an fbeta_score scoring object using make_scorer()
scorer = None

# TODO: Perform grid search on the classifier using 'scorer' as the scoring method using GridSearchCV()
grid_obj = None

# TODO: Fit the grid search object to the training data and find the optimal parameters using fit()
grid_fit = None

# Get the estimator
best_clf = grid_fit.best_estimator_

# Make predictions using the unoptimized and model
predictions = (clf.fit(X_train, y_train)).predict(X_test)
best_predictions = best_clf.predict(X_test)

# Report the before-and-afterscores
print("Unoptimized model\n------")
print("Accuracy score on testing data: {:.4f}".format(accuracy_score(y_test, predictions)))
print("F-score on testing data: {:.4f}".format(fbeta_score(y_test, predictions, beta = 0.5)))
print("\nOptimized Model\n------")
print("Final accuracy score on the testing data: {:.4f}".format(accuracy_score(y_test, best_predictions)))
print("Final F-score on the testing data: {:.4f}".format(fbeta_score(y_test, best_predictions, beta = 0.5)))

### Question 5 - Évaluation Finale du Modèle

* Quelle est l'exactitude et le F-score de votre modèle optimisé sur les données de test ?
* Ces scores sont-ils meilleurs ou pires que le modèle non optimisé ?
* Comment les résultats de votre modèle optimisé se comparent-ils aux références du prédicteur naïf que vous avez trouvées plus tôt dans la **Question 1** ?_

**Note :** Remplissez le tableau ci-dessous avec vos résultats, puis fournissez une discussion dans la case **Réponse**.

#### Résultats :

|     Métrique     | Modèle Non Optimisé | Modèle Optimisé |
| :--------------: | :-----------------: | :-------------: | 
| Score d'Exactitude |                   |                 |
| Score F          |                     |    EXEMPLE      |



**Réponse :**


----
## Importance des Caractéristiques

Une tâche importante lors de l'exécution d'un apprentissage supervisé sur un jeu de données comme les données de recensement que nous étudions ici est de déterminer quelles caractéristiques fournissent le plus de pouvoir prédictif. En se concentrant sur la relation entre seulement quelques caractéristiques cruciales et l'étiquette cible, nous simplifions notre compréhension du phénomène, ce qui est presque toujours utile. Dans le cas de ce projet, cela signifie que nous souhaitons identifier un petit nombre de caractéristiques qui prédisent le plus fortement si un individu gagne au plus ou plus de 50 000 $.

Choisissez un classificateur scikit-learn (par exemple, adaboost, forêts aléatoires) qui possède un attribut `feature_importance_`, qui est une fonction qui classe l'importance des caractéristiques selon le classificateur choisi. Dans la prochaine cellule Python, ajustez ce classificateur à l'ensemble d'entraînement et utilisez cet attribut pour déterminer les 5 caractéristiques les plus importantes pour le jeu de données du recensement.

### Question 6 - Observation de la Pertinence des Caractéristiques
Lors de **l'Exploration des Données**, il a été montré qu'il y a treize caractéristiques disponibles pour chaque individu enregistré dans les données du recensement. Parmi ces treize enregistrements, quelles sont selon vous les cinq caractéristiques les plus importantes pour la prédiction, dans quel ordre les classeriez-vous et pourquoi ?

**Réponse :**


### Implémentation - Extraction de l'Importance des Caractéristiques
Choisissez un algorithme d'apprentissage supervisé `scikit-learn` qui dispose d'un attribut `feature_importance_`. Cet attribut est une fonction qui classe l'importance de chaque caractéristique lors de la réalisation de prédictions basées sur l'algorithme choisi.

Dans la cellule de code ci-dessous, vous devrez implémenter ce qui suit :
 - Importer un modèle d'apprentissage supervisé de sklearn s'il est différent des trois utilisés précédemment.
 - Entraîner le modèle supervisé sur l'ensemble complet d'entraînement.
 - Extraire l'importance des caractéristiques en utilisant `'.feature_importances_'`.

In [ ]:
# TODO: Import a supervised learning model that has 'feature_importances_'


# TODO: Train the supervised model on the training set using .fit(X_train, y_train)
model = None

# TODO: Extract the feature importances using .feature_importances_ 
importances = None

# Plot
vs.feature_plot(importances, X_train, y_train)

### Question 7 - Extraction de l'Importance des Caractéristiques

Observez la visualisation créée ci-dessus qui affiche les cinq caractéristiques les plus pertinentes pour prédire si un individu gagne au plus ou plus de 50 000 $.
* Comment ces cinq caractéristiques se comparent-elles aux cinq caractéristiques dont vous avez discuté dans la **Question 6** ?
* Si vous étiez proche de la même réponse, comment cette visualisation confirme-t-elle vos réflexions ?
* Si vous n'étiez pas proche, pourquoi pensez-vous que ces caractéristiques sont plus pertinentes ?

**Réponse :**


### Sélection des Caractéristiques
Comment se comporte un modèle si nous n'utilisons qu'un sous-ensemble de toutes les caractéristiques disponibles dans les données ? Avec moins de caractéristiques à entraîner, on s'attend à ce que le temps d'entraînement et de prédiction soit beaucoup plus faible — au détriment des métriques de performance. D'après la visualisation ci-dessus, nous voyons que les cinq caractéristiques les plus importantes contribuent à plus de la moitié de l'importance de **toutes** les caractéristiques présentes dans les données. Cela suggère que nous pouvons essayer de *réduire l'espace des caractéristiques* et simplifier les informations requises pour l'apprentissage du modèle. La cellule de code ci-dessous utilisera le même modèle optimisé que vous avez trouvé précédemment, et l'entraînera sur le même ensemble d'entraînement *avec uniquement les cinq caractéristiques les plus importantes*.

In [ ]:
# Import functionality for cloning a model
from sklearn.base import clone

# Reduce the feature space
X_train_reduced = X_train[X_train.columns.values[(np.argsort(importances)[::-1])[:5]]]
X_test_reduced = X_test[X_test.columns.values[(np.argsort(importances)[::-1])[:5]]]

# Train on the "best" model found from grid search earlier
clf = (clone(best_clf)).fit(X_train_reduced, y_train)

# Make new predictions
reduced_predictions = clf.predict(X_test_reduced)

# Report scores from the final model using both versions of data
print("Final Model trained on full data\n------")
print("Accuracy on testing data: {:.4f}".format(accuracy_score(y_test, best_predictions)))
print("F-score on testing data: {:.4f}".format(fbeta_score(y_test, best_predictions, beta = 0.5)))
print("\nFinal Model trained on reduced data\n------")
print("Accuracy on testing data: {:.4f}".format(accuracy_score(y_test, reduced_predictions)))
print("F-score on testing data: {:.4f}".format(fbeta_score(y_test, reduced_predictions, beta = 0.5)))

### Question 8 - Effets de la Sélection des Caractéristiques

* Comment le F-score et le score d'exactitude du modèle final sur les données réduites utilisant uniquement cinq caractéristiques se comparent-ils à ces mêmes scores lorsque toutes les caractéristiques sont utilisées ?
* Si le temps d'entraînement était un facteur, envisageriez-vous d'utiliser les données réduites comme ensemble d'entraînement ?

**Réponse :**


**Note** : Une fois que vous avez terminé toutes les implémentations de code et répondu avec succès à chaque question ci-dessus, vous pouvez finaliser votre travail en exportant le Notebook iPython en tant que document HTML. Vous pouvez le faire en utilisant le menu en haut et en naviguant vers  
**Fichier -> Télécharger au format -> HTML (.html)**.
 
**Envoyez ensuite le document HTML sur: abdallah.elhidali@gmail.com**
